In [1]:
from HLsearch import *
from scipy.integrate import solve_ivp
import numpy as np
from sympy import symbols, var, diff, simplify, collect,solve
from sympy.utilities.lambdify import lambdify, implemented_function

from operator import add,sub,mul

import itertools
import scipy.io as sio

import time

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

def pendulum(t,x):
    return x[1],-9.81*np.sin(x[0])

t = np.arange(0,1,0.01)
y0=np.array([np.pi/4, 0])
X,Xdot = generate_data(pendulum,t,y0)

In [2]:
data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('x{}, x{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
expr = buildFunctionExpressions(2,X.shape[1],data_description,use_sine=True)
print(expr)
Theta = buildTimeSerieMatrixFromFunctions(X,expr, data_description)

Gamma = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr,data_description)


u, s, vh = np.linalg.svd(0*Theta-Gamma,full_matrices=False)

xi = vh.T[:,-1]

Variables are: (x0, x0_t)
['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


In [3]:
print('Now drop off small coefficients')
Hamiltonian,terms = generateSimplifiedExpression(xi,expr)

print('H = ',Hamiltonian)

Now drop off small coefficients
H =  0.021313435683958437*x0_t**2 - 0.4181696080660934*cos(x0)


In [4]:
print('Now start calculating CartPole')

g=9.81
mp=0.1
mc=1
l=1

def cartpole(t,y,f=1.0):
    theta,thetadot,x,xdot = y
    xdotdot = (f+mp*np.sin(theta)*(l*thetadot**2+g*np.cos(theta)))/(mc+mp*np.sin(theta)**2)
    thetadotdot = (-f*np.cos(theta)-mp*l*thetadot**2*np.cos(theta)*np.sin(theta)-(mc+mp)*g*np.sin(theta))/(l*(mc+mp*np.sin(theta)**2))
    return thetadot,thetadotdot,xdot,xdotdot

t = np.arange(0,0.5,0.01)
y0=np.array([np.pi/4, 0,0, 0])
X,Xdot = generate_data(cartpole,t,y0)


Now start calculating CartPole


In [5]:
data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('x{}, x{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
# print(data_description_sym)
# print([data_description[i] for i in range(round(len(data_description)/2))])

expr_new0 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2))],use_sine=True)
expr_new1 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2),len(data_description))],use_sine=False)

print(expr_new0[1:])
print(expr_new1)
expr_new = expr_new0[1:]+expr_new1
expr_new = buildFunctionExpressions(3,len(expr_new),expr_new)

Variables are: (x0, x0_t, x1, x1_t)
['x0_t', 'sin(x0)', 'cos(x0)']
['x1', 'x1_t']


In [6]:
Theta = buildTimeSerieMatrixFromFunctions(X,expr_new, data_description)
Gamma= buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr_new,data_description)
energyChange = 1.0*X[:,3]

In [7]:
stored_indices = tuple(expr_new.index(str(f)) for f in terms)

terms = [expr_new[i] for i in stored_indices]
print('Keeping terms: ',terms)
elements = tuple(x for x in range(len(expr_new)) if x not in stored_indices)
# Hamiltonian_old = Hamiltonian_old1+Hamiltonian_old2

# print(Hamiltonian_old)

combi_number = 2
indices = itertools.combinations(elements, combi_number)

Keeping terms:  ['cos(x0)', 'x0_t**2']


In [8]:
goodHamiltonian={}

def countNumberOfElementsLargerThanThreshold(x,threshold = 1e-12):
    count = 0
    for i in range(len(x)):
        if abs(x[i]) > threshold:
            count = count +1
    return count
start = time.time()

for count,index in enumerate(indices):
# index_tup = (2,5,16,25)
    index_tup = index + stored_indices
    xi, sumResidual = np.linalg.lstsq(Gamma[:,index_tup], energyChange,rcond=None)[:2]
    if sumResidual.size==0 or sumResidual>1e-12: continue
    if countNumberOfElementsLargerThanThreshold(xi)<=2: continue
    # if np.var(Gamma[:,index_tup]@xi-energyChange) > 1e-5: continue
#     xi = np.around(xi,decimals=12)
    expr_temp = [sympify(expr_new[i]) for i in index_tup]
    Hamiltonian = generateExpression(xi,expr_temp,threshold=1e-12)
#     print('Total Energy = ',Hamiltonian)
    Lagrangian = findLagrangianFromHamiltonian(Hamiltonian,expr_temp,data_description_sym,threshold=1e-12)
    if Lagrangian is not None and Lagrangian != '':
        goodHamiltonian[Hamiltonian] = Lagrangian
        print('Found good result at ',count,'th trial: ',index_tup)
print('Elapsed time: ',np.around(time.time()-start,2), 's')

for H,L in goodHamiltonian.items():
    print('')
    print('Hamiltonian is ',H)
    print('Lagrangian is ',L)  

Found good result at  771 th trial:  (19, 43, 2, 5)
Elapsed time:  0.18 s

Hamiltonian is  0.049999999999999545*x0_t**2 + 0.09999999999999967*x0_t*x1_t*cos(x0) + 0.5500000000000005*x1_t**2 - 0.980999999999992*cos(x0)
Lagrangian is  0.05*x0_t**2 + 0.1*x0_t*x1_t*cos(x0) + 0.55*x1_t**2 + 0.980999999999992*cos(x0)


In [9]:
from sympy.physics.mechanics import *
th,x = dynamicsymbols('theta x')
dth,dx = dynamicsymbols('theta x',1)
# print(dth,dx)
state = (th,dth,x,dx)
u = dynamicsymbols('u')
L_subbed = L.subs(list(zip(data_description_sym,state)))
# mprint(L_subbed)
N = ReferenceFrame('N')
P0 = Point("P0")
P0.set_vel(N,dx*N.x)
# A = N.orientnew('A', 'axis', [th, N.z])
# A.set_ang_vel(N, dth*N.z)
fl=[(P0,u*N.x)]
LM = LagrangesMethod(L_subbed,[th,x],forcelist=fl,frame=N)
me = LM.form_lagranges_equations()
mprint( simplify(me) )
equations = simplify(LM.rhs())
mprint(equations)

Matrix([
[ 0.980999999999992*sin(theta) + 0.1*cos(theta)*x'' + 0.1*theta''],
[-u - 0.1*sin(theta)*theta'**2 + 0.1*cos(theta)*theta'' + 1.1*x'']])
Matrix([
[                                                                                                                                          theta'],
[                                                                                                                                              x'],
[1.0*(1.0*u*cos(theta) + 10.7909999999999*sin(theta) + 0.05*sin(2*theta)*theta'**2 - 2.77555756156289e-17*sin(3*theta))/(0.1*cos(theta)**2 - 1.1)],
[                                                  -(1.0*u + 0.1*sin(theta)*theta'**2 + 0.490499999999996*sin(2*theta))/(0.1*cos(theta)**2 - 1.1)]])
